In [2]:
import pickle

import pandas as pd
import numpy as np

# Se leen los datos preprocesados

In [3]:
with open("../../data/bicycles/df_bicycles_preprocesed.pk1", "rb") as f:
    df_bicycles = pickle.load(f)
    
with open("../../data/weather/df_weather_preprocesed.pk1", "rb") as f:
    df_weather = pickle.load(f)    

# Me aseguro de que la columna que contiene la fecha es del tipo datetime

In [4]:
df_bicycles.dtypes

ride_id                      object
rideable_type                object
started_at           datetime64[ns]
ended_at             datetime64[ns]
start_station_id             object
end_station_id               object
member_casual                object
year                          int64
month                         int64
day                           int64
time_hms_ms         timedelta64[ns]
day_type                     object
event                          bool
dtype: object

In [5]:
df_weather.dtypes

DATE                      datetime64[ns]
temperature                      float64
wind_speed                       float64
precipitation                    float64
relative_humidity                float64
snow_depth                       float64
station_level_pressure           float64
year                               int32
month                              int32
day                                int32
time_hms_ms                       object
dtype: object

# Se realiza un merge con 'neares' para buscar la hora mas cercana

In [6]:
df_bicycles_sorted = df_bicycles.sort_values('started_at')
df_weather_sorted = df_weather.sort_values('DATE')

df_merged = pd.merge_asof(
    df_bicycles_sorted,
    df_weather_sorted,
    left_on='started_at',
    right_on='DATE',
    direction='nearest',
    tolerance=pd.Timedelta('1h') # Se ajusta lo que se considera el mas cercano
)

# Se seleccionan solo columnas numéricas (float o int)
weather_cols_numeric = df_weather.select_dtypes(include=['float64']).columns.tolist()

for col in weather_cols_numeric:
    mask_missing = df_merged[col].isna()
    
    # Media móvil
    df_merged.loc[mask_missing, col] = (
        df_merged[col].fillna(
            df_merged[col].rolling(window=10, min_periods=1).mean()
        )
    )

# Se realizan comprobaciones de los datos

In [ ]:
# Columnas a eliminar
cols_to_drop = ["DATE"]

# Crear una copia del DataFrame sin esas columnas
df_merged = df_merged.drop(columns=cols_to_drop).copy()


# Se comprueba el resutlado

In [10]:
df_merged.head(10)

,ride_id,rideable_type,started_at,ended_at,start_station_id,end_station_id,member_casual,year_x,month_x,day_x,...,temperature,wind_speed,precipitation,relative_humidity,snow_depth,station_level_pressure,year_y,month_y,day_y,time_hms_ms_y
0,98D355D9A9852BE9,classic_bike,2022-01-01 00:00:05,2022-01-01 00:01:48,623,623,casual,2022,1,1,...,7.8,2.1,0.0,83.0,0.0,980.8,2022.0,1.0,1.0,00:06:00.000
1,04706CA7F5BD25EE,electric_bike,2022-01-01 00:01:00,2022-01-01 00:04:39,13325,13137,casual,2022,1,1,...,7.8,2.1,0.0,83.0,0.0,980.8,2022.0,1.0,1.0,00:06:00.000
2,42178E850B92597A,electric_bike,2022-01-01 00:01:16,2022-01-01 00:32:14,TA1305000009,TA1305000009,casual,2022,1,1,...,7.8,2.1,0.0,83.0,0.0,980.8,2022.0,1.0,1.0,00:06:00.000
3,6B93C46E8F5B114C,classic_bike,2022-01-01 00:02:14,2022-01-01 00:31:07,623,623,casual,2022,1,1,...,7.8,2.1,0.0,83.0,0.0,980.8,2022.0,1.0,1.0,00:06:00.000
4,466943353EAC8022,classic_bike,2022-01-01 00:02:35,2022-01-01 00:31:04,623,623,casual,2022,1,1,...,7.8,2.1,0.0,83.0,0.0,980.8,2022.0,1.0,1.0,00:06:00.000
5,AC1F67BDCDDD5988,electric_bike,2022-01-01 00:03:36,2022-01-01 00:04:02,13431,13431,member,2022,1,1,...,7.8,2.1,0.0,83.0,0.0,980.8,2022.0,1.0,1.0,00:06:00.000
6,A5BD5A4FD53D5414,electric_bike,2022-01-01 00:05:59,2022-01-01 00:08:06,KA1503000038,TA1309000055,member,2022,1,1,...,7.8,2.1,0.0,83.0,0.0,980.8,2022.0,1.0,1.0,00:06:00.000
7,E93D5F426242BC48,electric_bike,2022-01-01 00:06:15,2022-01-01 00:13:24,TA1309000015,KA1503000022,casual,2022,1,1,...,7.8,2.1,0.0,83.0,0.0,980.8,2022.0,1.0,1.0,00:06:00.000
8,0C9545AFBACF60E1,classic_bike,2022-01-01 00:06:27,2022-01-01 00:09:06,18003,TA1305000003,member,2022,1,1,...,7.8,2.1,0.0,83.0,0.0,980.8,2022.0,1.0,1.0,00:06:00.000
9,22E39FF7ECA32D5E,classic_bike,2022-01-01 00:06:37,2022-01-01 00:16:44,TA1306000013,13017,casual,2022,1,1,...,7.8,2.1,0.0,83.0,0.0,980.8,2022.0,1.0,1.0,00:06:00.000


In [9]:
df_merged.describe()

,started_at,ended_at,year_x,month_x,day_x,time_hms_ms_x,DATE,temperature,wind_speed,precipitation,relative_humidity,snow_depth,station_level_pressure,year_y,month_y,day_y
count,9969971,9969971,9.969971e+06,9.969971e+06,9.969971e+06,9969971,9969971,9.969971e+06,9.969971e+06,9.969971e+06,9.969971e+06,9.969971e+06,9.969971e+06,9.969971e+06,9.969971e+06,9.969971e+06
mean,2023-03-13 02:04:12.545116416,2023-03-13 02:20:39.132660992,2.022689e+03,6.593871e+00,1.572793e+01,0 days 14:36:09.941747974,2023-03-13 02:04:12.057117696,1.739474e+01,4.353604e+00,6.754068e-02,5.871269e+01,3.005105e+01,9.933005e+02,2.022689e+03,6.593952e+00,1.572815e+01
min,2022-01-01 00:00:05,2022-01-01 00:01:48,2.022000e+03,1.000000e+00,1.000000e+00,0 days 00:00:00,2022-01-01 00:06:00,-2.280000e+01,0.000000e+00,0.000000e+00,1.507547e+01,0.000000e+00,9.586000e+02,2.022000e+03,1.000000e+00,1.000000e+00
25%,2022-08-04 18:02:25.500000,2022-08-04 18:19:17,2.022000e+03,5.000000e+00,8.000000e+00,0 days 11:12:56,2022-08-04 18:02:00,1.060000e+01,2.866667e+00,0.000000e+00,4.523333e+01,3.000000e+01,9.893600e+02,2.022000e+03,5.000000e+00,8.000000e+00
50%,2023-04-12 17:11:21,2023-04-12 17:27:46,2.023000e+03,7.000000e+00,1.600000e+01,0 days 15:26:42,2023-04-12 17:11:00,1.940000e+01,4.184906e+00,0.000000e+00,5.891071e+01,3.000000e+01,9.932283e+02,2.023000e+03,7.000000e+00,1.600000e+01
75%,2023-09-08 12:05:13,2023-09-08 12:18:53.500000,2.023000e+03,9.000000e+00,2.300000e+01,0 days 18:08:37,2023-09-08 12:05:00,2.494000e+01,5.701887e+00,0.000000e+00,7.225000e+01,3.000000e+01,9.970550e+02,2.023000e+03,9.000000e+00,2.300000e+01
max,2024-05-31 23:59:47,2024-06-01 17:56:00,2.024000e+03,1.200000e+01,3.100000e+01,0 days 23:59:59,2024-06-01 00:00:00,3.830000e+01,1.650000e+01,3.930000e+01,1.000000e+02,2.300000e+02,1.016800e+03,2.024000e+03,1.200000e+01,3.100000e+01
std,NaN,NaN,6.846879e-01,2.746763e+00,8.774949e+00,0 days 04:52:43.867316925,NaN,9.627336e+00,2.177061e+00,6.027136e-01,1.735138e+01,7.557259e+00,6.276062e+00,6.846853e-01,2.746761e+00,8.774956e+00


In [7]:
df_merged.isna().sum()

ride_id                   0
rideable_type             0
started_at                0
ended_at                  0
start_station_id          0
end_station_id            0
member_casual             0
year_x                    0
month_x                   0
day_x                     0
time_hms_ms_x             0
day_type                  0
event                     0
DATE                      0
temperature               0
wind_speed                0
precipitation             0
relative_humidity         0
snow_depth                0
station_level_pressure    0
year_y                    0
month_y                   0
day_y                     0
time_hms_ms_y             0
dtype: int64

In [10]:
df_merged.dtypes

ride_id                            object
rideable_type                      object
started_at                 datetime64[ns]
ended_at                   datetime64[ns]
start_station_id                   object
end_station_id                     object
member_casual                      object
year_x                              int64
month_x                             int64
day_x                               int64
time_hms_ms_x             timedelta64[ns]
day_type                           object
event                                bool
DATE                       datetime64[ns]
temperature                       float64
wind_speed                        float64
precipitation                     float64
relative_humidity                 float64
snow_depth                        float64
station_level_pressure            float64
year_y                              int32
month_y                             int32
day_y                               int32
time_hms_ms_y                     

# Se guardan los datos

In [12]:
df_merged.to_pickle("../../data/join/df_data.pk1")